In [47]:
%pip install spotipy pandas

Note: you may need to restart the kernel to use updated packages.


In [48]:
%pip install spotipy pandas lyricsgenius

Note: you may need to restart the kernel to use updated packages.


In [55]:
import os
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import lyricsgenius
import re

def load_spotify_credentials(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            credentials[key] = value
    return credentials

# 프로젝트 루트 디렉토리 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# credentials.txt 파일 경로 설정
credentials_file = os.path.join(project_root, 'spotify_credentials.txt')

# 인증 정보 불러오기
credentials = load_spotify_credentials(credentials_file)
client_id = credentials['client_id']
client_secret = credentials['client_secret']

# Spotify API 인증 객체 생성
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [56]:
# Genius API 설정
genius = lyricsgenius.Genius("your_genius_api_key")

In [57]:
# 최신 노래 데이터 수집 함수
def get_new_releases(limit=1000):
    results = sp.new_releases(country='US', limit=limit)
    albums = [{'name': album['name'], 'artist': album['artists'][0]['name']} for album in results['albums']['items']]
    return albums

# 가사 수집 함수
def get_lyrics(song_title, artist_name):
    try:
        song = genius.search_song(song_title, artist_name)
        return song.lyrics if song else ""
    except Exception as e:
        return ""
    
# 파일 이름 정리 함수
def clean_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '', filename)

In [58]:
# 데이터 수집 및 가사 파일 저장
albums = get_new_releases(limit=50)
music_dir = os.path.join(project_root, 'data', 'music')
os.makedirs(music_dir, exist_ok=True)

for album in albums:
    lyrics = get_lyrics(album['name'], album['artist'])
    file_name = clean_filename(f"{album['name']}.txt")
    file_path = os.path.join(music_dir, file_name)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(lyrics)

Searching for "Louder" by Elephant Kind...
Done.
Searching for "Roadtrip" by Ardhito Pramono...
No results found for: 'Roadtrip Ardhito Pramono'
Searching for "CANDYRELLA" by Paul Partohap...
No results found for: 'CANDYRELLA Paul Partohap'
Searching for "Give All" by NUZB...
Done.
Searching for "I Believe" by Bassjackers...
Done.
Searching for "Both Ways" by The Script...
Done.
Searching for "High On Love" by Armin van Buuren...
Done.
Searching for "Chase It (Mmm Da Da Da)" by Bebe Rexha...
Done.
Searching for "Save My Soul (feat. Jordan Grace)" by Afroki...
Done.
Searching for "My City (feat. Katy Tiz)" by Tiësto...
Done.
Searching for "May flowers" by LIU KOI...
Done.
Searching for "Drive ft. KIDDO" by Steve Aoki...
Done.
Searching for "All the Things She Said" by Robin Schulz...
Done.
Searching for "Day 0 (feat. Dame D.O.L.L.A.)" by MIRROR...
Done.
Searching for "Devotion" by KSHMR...
Done.
Searching for "waiting for you" by Jasmine Kelly...
Done.
Searching for "Insomnia" by Eve...

In [59]:
# DataFrame으로 변환 및 CSV 파일로 저장 (노래 제목과 아티스트 정보만 저장)
df = pd.DataFrame(albums)
output_path = os.path.join(project_root, 'data', 'new_releases.csv')
df.to_csv(output_path, index=False)